In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
#plotting
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.gridspec as gridspec
from matplotlib.ticker import MaxNLocator


#sklearn
from sklearn.model_selection import cross_val_score,train_test_split

#stats
from scipy import stats
from scipy.stats import skew, boxcox_normmax, norm
from scipy.special import boxcox1p
from statsmodels.stats.outliers_influence import variance_inflation_factor


from sklearn.preprocessing import StandardScaler
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
plt.style.use('fivethirtyeight')
%matplotlib inline

In [ ]:
df=pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/train_data.csv')
df1=pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/test_data.csv')

In [ ]:
# none_cols=['trip_creation_time','route_schedule_uuid','cutoff_timestamp','od_end_time','destination_center']
# zero_cols=['osrm_distance','osrm_time','factor','segment_factor']
# # for col in zero_cols:
# #     df[col].replace(np.nan, 0, inplace=True)

# for col in none_cols:
#     df[col].replace(np.nan, 'None', inplace=True)

In [ ]:
print(df.isna().sum())
print(df1.isna().sum())

UID                                   0
trip_creation_time                 3515
route_schedule_uuid                2895
route_type                            0
trip_uuid                             0
source_center                         0
source_name                           0
destination_center                  803
destination_name                      0
od_start_time                         0
od_end_time                         720
start_scan_to_end_scan                0
is_cutoff                             0
cutoff_factor                         0
cutoff_timestamp                    930
actual_distance_to_destination        0
osrm_time                         14432
osrm_distance                     14432
factor                             1644
segment_actual_time                   0
segment_osrm_time                     0
segment_osrm_distance                 0
segment_factor                     1159
actual_time                           0
dtype: int64
UID                        

In [ ]:
nullcols = [col for col in df.columns if df[col].isnull().sum() >120000]
df1.drop(nullcols, axis=1, inplace=True)
df.drop(nullcols, axis=1, inplace=True)

In [ ]:
df.head()

,UID,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time
0,lWYYrbNEdf,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0
1,iNJcHQlPCm,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,902.759458,True,22,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0
2,U6N0M43qwR,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0
3,RrJUneuYNj,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0
4,7QLdD7iRC9,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,1109.682268,True,198,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0


In [ ]:
df.shape[1]

24

In [ ]:
df1.shape[1]

23

In [ ]:
df = df.drop('UID', axis=1)
df1 = df1.drop('UID', axis=1)

In [ ]:
df.head()
df.info()
df['od_start_time1']=df['od_start_time']
df['od_end_time1']=df['od_end_time']
df['trip_creation_time1']=df['trip_creation_time']
df1['od_start_time1']=df1['od_start_time']
df1['od_end_time1']=df1['od_end_time']
df1['trip_creation_time1']=df1['trip_creation_time']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 23 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   trip_creation_time              68643 non-null  object 
 1   route_schedule_uuid             69263 non-null  object 
 2   route_type                      72158 non-null  object 
 3   trip_uuid                       72158 non-null  object 
 4   source_center                   72158 non-null  object 
 5   source_name                     72158 non-null  object 
 6   destination_center              71355 non-null  object 
 7   destination_name                72158 non-null  object 
 8   od_start_time                   72158 non-null  object 
 9   od_end_time                     71438 non-null  object 
 10  start_scan_to_end_scan          72158 non-null  float64
 11  is_cutoff                       72158 non-null  bool   
 12  cutoff_factor                   

In [ ]:

df['od_start_time'] = pd.to_datetime(df['od_start_time'])
df['od_end_time'] = pd.to_datetime(df['od_end_time'])
df['segment_key'] = df['trip_uuid'] + df['source_center'] + df['destination_center']

segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df[col + '_sum'] = df.groupby('segment_key')[col].cumsum()

df[[col + '_sum' for col in segment_cols]]
df.info()
df1['od_start_time'] = pd.to_datetime(df1['od_start_time'])
df1['od_end_time'] = pd.to_datetime(df1['od_end_time'])
df1['segment_key'] = df1['trip_uuid'] + df1['source_center'] + df1['destination_center']

segment_cols = ['segment_actual_time', 'segment_osrm_distance', 'segment_osrm_time']

for col in segment_cols:
    df1[col + '_sum'] = df1.groupby('segment_key')[col].cumsum()

df1[[col + '_sum' for col in segment_cols]]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 30 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              68643 non-null  object        
 1   route_schedule_uuid             69263 non-null  object        
 2   route_type                      72158 non-null  object        
 3   trip_uuid                       72158 non-null  object        
 4   source_center                   72158 non-null  object        
 5   source_name                     72158 non-null  object        
 6   destination_center              71355 non-null  object        
 7   destination_name                72158 non-null  object        
 8   od_start_time                   72158 non-null  datetime64[ns]
 9   od_end_time                     71438 non-null  datetime64[ns]
 10  start_scan_to_end_scan          72158 non-null  float64       
 11  is

,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum
0,16.0,10.8152,7.0
1,22.0,14.7305,12.0
2,15.0,12.1171,11.0
3,25.0,23.4819,21.0
4,38.0,26.8622,24.0
...,...,...,...
72153,44.0,15.3398,23.0
72154,56.0,32.3240,49.0
72155,69.0,47.6151,70.0
72156,81.0,55.8009,82.0


In [ ]:
df['od_time_diff_hour'] = (df['od_end_time'] - df['od_start_time']).dt.total_seconds() /(60)
df['od_time_diff_hour']
df1['od_time_diff_hour'] = (df1['od_end_time'] - df1['od_start_time']).dt.total_seconds() /(60)
df1['od_time_diff_hour']
df.head()

,trip_creation_time,route_schedule_uuid,route_type,trip_uuid,source_center,source_name,destination_center,destination_name,od_start_time,od_end_time,start_scan_to_end_scan,is_cutoff,cutoff_factor,cutoff_timestamp,actual_distance_to_destination,osrm_time,osrm_distance,factor,segment_actual_time,segment_osrm_time,segment_osrm_distance,segment_factor,actual_time,od_start_time1,od_end_time1,trip_creation_time1,segment_key,segment_actual_time_sum,segment_osrm_distance_sum,segment_osrm_time_sum,od_time_diff_hour
0,2018-09-15 02:23:18.656742,thanos::sroute:9d8ed3f7-a8af-4393-ad7a-c57a720...,FTL,trip-153697819865649878,IND832109AAB,Jamshedpur_Central_I_3 (Jharkhand),IND834002AAB,Ranchi_Hub (Jharkhand),2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,713.521495,True,44,2018-09-15 04:45:23,158.807515,778.672716,1228.735753,2.782826,77.988686,20.793472,28.475783,3.292773,100.0,2018-09-15 02:23:18.656742,2018-09-15 06:33:35.824186,2018-09-15 02:23:18.656742,trip-153697819865649878IND832109AABIND834002AAB,77.988686,28.475783,20.793472,250.286124
1,2018-09-30 15:07:48.971326,thanos::sroute:b9142a73-d68f-46ec-9afc-d145479...,FTL,trip-153832006897106333,IND422011AAD,Nashik_TgrniaRD_I (Maharashtra),IND421302AAG,Bhiwandi_Mankoli_HB (Maharashtra),2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,902.759458,True,22,2018-10-01 05:38:30,81.388109,227.479029,NaN,3.625248,68.517914,18.787231,34.612259,4.277350,61.0,2018-09-30 22:38:13.903069,2018-10-01 06:48:37.637477,2018-09-30 15:07:48.971326,trip-153832006897106333IND422011AADIND421302AAG,68.517914,34.612259,18.787231,490.395573
2,2018-09-23 22:15:46.635975,thanos::sroute:883e99fa-50a3-40e0-a2e2-9b12ed6...,FTL,trip-153774094663572416,IND845305AAA,Raxaul_KairiyaT_D (Bihar),NaN,Muzaffrpur_Bbganj_I (Bihar),2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,923.666146,True,22,2018-09-24 09:21:30,134.486898,400.072069,1188.943158,4.415969,70.475386,24.371978,29.867950,5.322072,69.0,2018-09-24 03:56:37.016232,2018-09-24 11:15:00.826140,2018-09-23 22:15:46.635975,NaN,NaN,NaN,NaN,438.396832
3,2018-09-21 05:11:44.833176,thanos::sroute:96a80600-40e1-436b-9161-fa68f9e...,FTL,trip-153750670483274503,IND842001AAA,Muzaffrpur_Bbganj_I (Bihar),IND854326AAB,Purnia_Central_H_2 (Bihar),2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,705.451567,True,176,2018-09-22 14:09:17,262.866674,510.123878,999.421258,2.946647,101.876844,19.588977,27.794019,4.265799,300.0,2018-09-22 10:08:32.593022,2018-09-22 19:17:22.242888,2018-09-21 05:11:44.833176,trip-153750670483274503IND842001AAAIND854326AAB,101.876844,27.794019,19.588977,548.827498
4,2018-09-18 01:10:49.936747,thanos::sroute:b43ec86d-59c1-452c-b4a9-807d357...,FTL,trip-153723305402762987,IND411033AAA,Pune_Tathawde_H (Maharashtra),IND501359AAE,Hyderabad_Shamshbd_H (Telangana),2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,1109.682268,True,198,2018-09-18 12:14:05,379.389280,NaN,NaN,1.998790,46.571301,22.671495,26.971844,3.553652,286.0,2018-09-18 01:10:49.936747,2018-09-18 17:58:48.449251,2018-09-18 01:10:49.936747,trip-153723305402762987IND411033AAAIND501359AAE,46.571301,26.971844,22.671495,1007.975208


In [ ]:
def place2state(x):
    # transform "gurgaon_bilaspur_hb (haryana)" into "haryana"
    state = x.split('(')[1]

    return state[:-1] #removing ')' from ending

def place2city(x):
    #we will remove state
    city = x.split(' (')[0]

    city = city.split('_')[0]

    # Now daling with edge cases

    if city == 'pnq vadgaon sheri dpc': return 'vadgaonsheri'

    # ['PNQ Pashan DPC', 'Bhopal MP Nagar', 'HBR Layout PC',
    #  'PNQ Rahatani DPC', 'Pune Balaji Nagar', 'Mumbai Antop Hill']

    if city in ['pnq pashan dpc','pnq rahatani dpc', 'pune balaji nagar']:
        return 'pune'

    if city == 'hbr layout pc' :
        return 'bengaluru'
    if city == 'bhopal mp nagar':
        return 'bhopal'
    if city == 'mumbai antop hill':
        return 'mumbai'

    return city

def place2city_place(x):

    # we will remove state
    x = x.split('(')[0]

    len_ = len(x.split('_'))

    if len_ >= 3:
        return x.split('_')[1]

    # small cities have same city and place name
    if len_ == 2:
        return x.split('_')[0]

    # now we need to deal with edge cases or imporper name convention

    # if len(x.split('_')) == 2:

    return x.split(' ')[0]

def place2code(x):
    # we will remove state
    x = x.split('(')[0]

    if len(x.split('_')) >= 3:
        return x.split('_')[-1]

    return 'none'

In [ ]:
df['destination_state'] = df['destination_name'].apply(lambda x: place2state(x))
df['destination_city']  = df['destination_name'].apply(lambda x: place2city(x))
df['destination_place'] = df['destination_name'].apply(lambda x: place2city_place(x))
df['destination_code']  = df['destination_name'].apply(lambda x: place2code(x))
df1['destination_state'] = df1['destination_name'].apply(lambda x: place2state(x))
df1['destination_city']  = df1['destination_name'].apply(lambda x: place2city(x))
df1['destination_place'] = df1['destination_name'].apply(lambda x: place2city_place(x))
df1['destination_code']  = df1['destination_name'].apply(lambda x: place2code(x))

In [ ]:
df['trip_creation_time'] = pd.to_datetime(df['trip_creation_time'])

df['trip_year'] = df['trip_creation_time'].dt.year
df['trip_month'] = df['trip_creation_time'].dt.month
df['trip_hour'] = df['trip_creation_time'].dt.hour
df['trip_day'] = df['trip_creation_time'].dt.day
df['trip_week'] = df['trip_creation_time'].dt.isocalendar().week
df['trip_dayofweek'] = df['trip_creation_time'].dt.dayofweek
df1['trip_creation_time'] = pd.to_datetime(df1['trip_creation_time'])

df1['trip_year'] = df1['trip_creation_time'].dt.year
df1['trip_month'] = df1['trip_creation_time'].dt.month
df1['trip_hour'] = df1['trip_creation_time'].dt.hour
df1['trip_day'] = df1['trip_creation_time'].dt.day
df1['trip_week'] = df1['trip_creation_time'].dt.isocalendar().week
df1['trip_dayofweek'] = df1['trip_creation_time'].dt.dayofweek

In [ ]:
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','segment_actual_time_sum','osrm_time','osrm_distance','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']

In [ ]:
df['route_type'] = df['route_type'].map({'FTL':0, 'Carting':1})
df1['route_type'] = df1['route_type'].map({'FTL':0, 'Carting':1})

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[num_cols])
df[num_cols] = scaler.transform(df[num_cols])
num_cols = ['start_scan_to_end_scan','actual_distance_to_destination','segment_actual_time_sum','osrm_time','osrm_distance','segment_osrm_distance_sum',
           'segment_osrm_time_sum', 'od_time_diff_hour']
scaler = StandardScaler()
scaler.fit(df1[num_cols])
df1[num_cols] = scaler.transform(df1[num_cols])

In [ ]:
df.head()
df.info()
df1.info()

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              68643 non-null  datetime64[ns]
 1   route_schedule_uuid             69263 non-null  object        
 2   route_type                      72158 non-null  int64         
 3   trip_uuid                       72158 non-null  object        
 4   source_center                   72158 non-null  object        
 5   source_name                     72158 non-null  object        
 6   destination_center              71355 non-null  object        
 7   destination_name                72158 non-null  object        
 8   od_start_time                   72158 non-null  datetime64[ns]
 9   od_end_time                     71438 non-null  datetime64[ns]
 10  start_scan_to_end_scan          72158 non-null  float64       
 11  is

In [ ]:
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
# for col in df.columns:
#     if df[col].dtype == 'object' or df[col].dtype == 'bool':
#         le = ce.HashingEncoder(n_components=1)
#         df[col] = le.fit_transform(df[col])

# for col in df1.columns:
#     if df1[col].dtype == 'object' or df1[col].dtype == 'bool':
#         le = ce.HashingEncoder(n_components=1)
#         df1[col] = le.fit_transform(df1[col])
for col in df.columns:
    if df[col].dtype == 'object' or df[col].dtype == 'bool':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

for col in df1.columns:
    if df1[col].dtype == 'object' or df1[col].dtype == 'bool':
        le = LabelEncoder()
        df1[col] = le.fit_transform(df1[col])
# encoder_purpose = ce.HashingEncoder(n_components=6)
# encoder_purpose.fit_transform(df[])
# encoder_purpose.fit_transform(df1)

In [ ]:
df.head()
df.isnull().sum()
df.info()
df.drop(['trip_week','trip_creation_time','od_start_time','od_end_time'],axis=1,inplace=True)
df1.drop(['trip_week','trip_creation_time','od_start_time','od_end_time'],axis=1,inplace=True)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 41 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   trip_creation_time              68643 non-null  datetime64[ns]
 1   route_schedule_uuid             72158 non-null  int64         
 2   route_type                      72158 non-null  int64         
 3   trip_uuid                       72158 non-null  int64         
 4   source_center                   72158 non-null  int64         
 5   source_name                     72158 non-null  int64         
 6   destination_center              72158 non-null  int64         
 7   destination_name                72158 non-null  int64         
 8   od_start_time                   72158 non-null  datetime64[ns]
 9   od_end_time                     71438 non-null  datetime64[ns]
 10  start_scan_to_end_scan          72158 non-null  float64       
 11  is

In [ ]:
df.fillna(df.mean(), inplace=True)

df1.fillna(df1.mean(), inplace=True)

In [ ]:
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 37 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   route_schedule_uuid             72158 non-null  int64  
 1   route_type                      72158 non-null  int64  
 2   trip_uuid                       72158 non-null  int64  
 3   source_center                   72158 non-null  int64  
 4   source_name                     72158 non-null  int64  
 5   destination_center              72158 non-null  int64  
 6   destination_name                72158 non-null  int64  
 7   start_scan_to_end_scan          72158 non-null  float64
 8   is_cutoff                       72158 non-null  int64  
 9   cutoff_factor                   72158 non-null  int64  
 10  cutoff_timestamp                72158 non-null  int64  
 11  actual_distance_to_destination  72158 non-null  float64
 12  osrm_time                       

In [ ]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

for col in df.columns:
    replace_with_thresholds(df, col)

for col in df1.columns:
    replace_with_thresholds(df1, col)


In [ ]:
cor = df.corr()

In [ ]:
X = df.drop(["actual_time"],1)   #Feature Matrix
y = df["actual_time"]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
names = X.columns
indexes = X.index
sc = StandardScaler()
X = sc.fit_transform(X)
X = pd.DataFrame(X, columns=names, index=indexes)

names = df1.columns
indexes = df1.index
sc = StandardScaler()
df1 = sc.fit_transform(df1)
df1 = pd.DataFrame(df1, columns=names, index=indexes)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.001, random_state=42)
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix

In [ ]:
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from lightgbm import LGBMRegressor
nn_model=CatBoostRegressor(n_estimators=20000,learning_rate=0.075,max_depth=9)
nn_model.fit(X_train,y_train)

0:	learn: 558.4915856	total: 60.5ms	remaining: 2m 1s
1:	learn: 520.7451145	total: 100ms	remaining: 1m 40s
2:	learn: 485.1881176	total: 138ms	remaining: 1m 31s
3:	learn: 452.4223956	total: 177ms	remaining: 1m 28s
4:	learn: 421.7598386	total: 215ms	remaining: 1m 25s
5:	learn: 393.3459767	total: 252ms	remaining: 1m 23s
6:	learn: 367.4727305	total: 291ms	remaining: 1m 22s
7:	learn: 342.9419561	total: 330ms	remaining: 1m 22s
8:	learn: 320.7055783	total: 367ms	remaining: 1m 21s
9:	learn: 300.1137983	total: 403ms	remaining: 1m 20s
10:	learn: 281.3181295	total: 440ms	remaining: 1m 19s
11:	learn: 263.5719577	total: 478ms	remaining: 1m 19s
12:	learn: 247.0848221	total: 516ms	remaining: 1m 18s
13:	learn: 232.0476251	total: 553ms	remaining: 1m 18s
14:	learn: 218.3449878	total: 590ms	remaining: 1m 18s
15:	learn: 205.5302331	total: 625ms	remaining: 1m 17s
16:	learn: 194.0729708	total: 662ms	remaining: 1m 17s
17:	learn: 182.9138839	total: 697ms	remaining: 1m 16s
18:	learn: 172.9200924	total: 734ms	re

In [ ]:
y_val_pred = nn_model.predict(X_val)

In [ ]:
X_test = df1
X_test = X_test[X_train.columns]

In [ ]:
rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print(f"Root Mean Squared Error: {rmse}")

Root Mean Squared Error: 24.583502191518598


In [ ]:
y_pred = nn_model.predict(X_test)
X_test.info()
y_pred

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72158 entries, 0 to 72157
Data columns (total 36 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   route_schedule_uuid             72158 non-null  float64
 1   route_type                      72158 non-null  float64
 2   trip_uuid                       72158 non-null  float64
 3   source_center                   72158 non-null  float64
 4   source_name                     72158 non-null  float64
 5   destination_center              72158 non-null  float64
 6   destination_name                72158 non-null  float64
 7   start_scan_to_end_scan          72158 non-null  float64
 8   is_cutoff                       72158 non-null  float64
 9   cutoff_factor                   72158 non-null  float64
 10  cutoff_timestamp                72158 non-null  float64
 11  actual_distance_to_destination  72158 non-null  float64
 12  osrm_time                       

array([ 44.59740702,  66.60135933,  16.45121222, ...,  62.92381936,
        94.48307534, 138.53718148])

In [ ]:
output_df = pd.read_csv('/kaggle/input/cassandra23-ps-2/dataset/sample_submissions.csv')
submissions = pd.DataFrame({'UID': output_df['UID'], 'actual_time': (y_pred).astype(float)})

In [ ]:
submissions.to_csv('submissions.csv', index=False)

In [ ]:
submissions

,UID,actual_time
0,XMsT9clviB,44.597407
1,1iUuyRUZ2Y,66.601359
2,wancwiAIU3,16.451212
3,1tIt8EwJjB,64.836852
4,W2vL6bJrjt,36.065873
...,...,...
72153,L5IrEpNBFb,43.132605
72154,s5TDXjnN3s,49.785971
72155,0croXy5yK5,62.923819
72156,P0okvBa2Ik,94.483075
